In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D,Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant


In [ ]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

--2020-08-06 14:04:42--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla [following]
--2020-08-06 14:04:42--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf52013098368a7d5348f6f839c.dl.dropboxusercontent.com/zip_download_get/AgVIIugjenZWjGSvDPFJoYAkXRhtns8vcWlvCmP-ZXvtIjtLQna2S7H4YsASYcWOzjsrghvYg_WKK_G23bB5yzp_-WonMpV6_KLiz19U8s8-Pg [following]
--2020-08-06 14:04:43--  https://ucf52013098368a7d5348f6f839c.dl.dropboxusercontent.com/zip_download_get/AgVIIugjenZWjGSvDPFJoYAkXRhtns8vcWlvCmP-ZXvtIjtLQna2S7H4YsASYcWOzjsrghvYg_WKK_G23bB5

In [ ]:
!unzip '/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0'


Archive:  /content/AAAw1_JIzpuVvwteJCma0xMla?dl=0
mapname:  conversion of  failed
 extracting: train.csv               

1 archive had fatal errors.


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2020-08-06 14:04:50--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-08-06 14:04:50--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-08-06 14:04:50--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
!unzip '/content/glove.6B.zip'


Archive:  /content/glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
ds = r'/content/train.csv'


In [ ]:
d = pd.read_csv(ds)

In [ ]:
d

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0
...,...,...,...
1306117,ffffcc4e2331aaf1e41e,What other technical skills do you need as a c...,0
1306118,ffffd431801e5a2f4861,Does MS in ECE have good job prospects in USA ...,0
1306119,ffffd48fb36b63db010c,Is foam insulation toxic?,0
1306120,ffffec519fa37cf60c78,How can one start a research project based on ...,0


In [ ]:
d.drop(['qid'],axis=1,inplace=True)

In [ ]:
d

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0
...,...,...
1306117,What other technical skills do you need as a c...,0
1306118,Does MS in ECE have good job prospects in USA ...,0
1306119,Is foam insulation toxic?,0
1306120,How can one start a research project based on ...,0


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(d['question_text'])
sequences = tokenizer.texts_to_sequences(d['question_text'])

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 222161 unique tokens.


In [ ]:
max_len = 0
i= 0 
for i in range(len(sequences)):
    if len(sequences[i]) > max_len:
        max_len = len(sequences[i])
        #print(max_len)
        #print(sequences[i])
print(max_len)

190


In [ ]:

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X_data = pad_sequences(sequences, maxlen=max_len)

Found 222161 unique tokens.


In [ ]:

labels = to_categorical(np.asarray(d['target']))

In [ ]:
print(labels.shape)
print(X_data.shape)

(1306122, 2)
(1306122, 190)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
glove_file = r'/content/glove.6B.100d.txt'
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [ ]:
indices = np.arange(X_data.shape[0])
np.random.shuffle(indices)
X_data = X_data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X_data.shape[0])
print(nb_validation_samples)

x_train = X_data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = X_data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

261224


In [ ]:
x_train

array([[     0,      0,      0, ...,     19,   1498,  14743],
       [     0,      0,      0, ...,      5,   8464, 129995],
       [     0,      0,      0, ...,      8,     98,    453],
       ...,
       [     0,      0,      0, ...,    218,    383,   8476],
       [     0,      0,      0, ...,      6,     94,   5350],
       [     0,      0,      0, ...,      6,   8636,  31332]], dtype=int32)

In [ ]:
y_train


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
embeddings_index = {}
f = glove_file
f = open(f,encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

In [ ]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)

In [ ]:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
#x = Conv1D(128, 5, activation='relu')(x)
#x = MaxPooling1D(5)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(2, activation='sigmoid')(x)

model = Model(sequence_input, outputs)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])
#fitting
model.fit(x_train, y_train, validation_data=(x_val,y_val),
          epochs=5, batch_size=128)

Epoch 1/5
8164/8164 [==============================] - 1756s 215ms/step - loss: 0.2019 - acc: 0.9427 - val_loss: 0.1982 - val_acc: 0.9432
Epoch 2/5
8164/8164 [==============================] - 1766s 216ms/step - loss: 0.1974 - acc: 0.9439 - val_loss: 0.1974 - val_acc: 0.9438
Epoch 3/5
8164/8164 [==============================] - 1810s 222ms/step - loss: 0.1953 - acc: 0.9446 - val_loss: 0.1975 - val_acc: 0.9437
Epoch 4/5
8164/8164 [==============================] - 1775s 217ms/step - loss: 0.1931 - acc: 0.9454 - val_loss: 0.1986 - val_acc: 0.9427
Epoch 5/5
8164/8164 [==============================] - 1805s 221ms/step - loss: 0.1909 - acc: 0.9465 - val_loss: 0.1990 - val_acc: 0.9437


In [ ]:
       y_pred = model.predict(x_val)

In [ ]:

y_pred [y_pred > 0.5] = 1.0
y_pred[y_pred <= 0.5] = 0.0

In [ ]:
y_pred

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
pred = pd.DataFrame(y_pred)


In [ ]:
pred.to_csv('predctions.csv',index=False)


In [ ]:
model.save_weights('spam filter_que')